In [1]:
using GeometryTypes
const gt = GeometryTypes
using LCMGL
using Meshing
using DrakeVisualizer
using BenchmarkTools

adding: /Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [42]:
include("gjk.jl")

Gjk

In [43]:
s = Simplex(Vec(2., -1), Vec(2.5, 0), Vec(2., 1), Vec(1., 1), Vec(1., -1))
simplex, best_pt = Gjk.gjk(s)
@show best_pt
simplex._

length(simplex) = 1
N = 2
length(simplex) = 2
N = 2
best_pt = Vec(1.0,0.0)


3-element Array{Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}},1}:
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(2.0,-1.0),Vec(2.0,-1.0))
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,1.0),Vec(1.0,1.0))  
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,-1.0),Vec(1.0,-1.0))

In [44]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
acc = Gjk.AcceleratedMesh(mesh)
pt = Vec(0., 0, 0)
md = gt.MinkowskiDifference(acc, pt)
simplex, best_pt = Gjk.gjk(md)
@show best_pt
simplex

length(simplex) = 1
N = 3
length(simplex) = 2
N = 3
length(simplex) = 3
N = 3
best_pt = Vec(1.2000000000000002,-0.8999999999999999,0.0)


GeometryTypes.FlexibleSimplex{Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}}(Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}[Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.2,-1.5,-0.6),(1,Vec(0.0,0.0,0.0))),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.41522,-0.75,0.3),(291,Vec(0.0,0.0,0.0))),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.12286,-1.05,-0.15),(45,Vec(0.0,0.0,0.0))),Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.2,-0.9,0.0),(253,Vec(0.0,0.0,0.0)))])

In [45]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
simplex = Gjk.gjk(mesh)[1]
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex._
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

length(simplex) = 1
N = 3
length(simplex) = 2
N = 3
length(simplex) = 3
N = 3


In [28]:
acc = Gjk.AcceleratedMesh(mesh)
simplex, pt_best = Gjk.gjk(acc)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex._
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [20]:
@benchmark Gjk.gjk($mesh)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  4.84 kb
  allocs estimate:  103
  minimum time:     11.80 μs (0.00% GC)
  median time:      12.92 μs (0.00% GC)
  mean time:        22.38 μs (33.45% GC)
  maximum time:     74.95 ms (99.88% GC)

In [21]:
@benchmark Gjk.gjk($acc)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  5.61 kb
  allocs estimate:  135
  minimum time:     12.09 μs (0.00% GC)
  median time:      12.80 μs (0.00% GC)
  mean time:        13.81 μs (5.67% GC)
  maximum time:     4.37 ms (97.67% GC)

In [22]:
simplex, pt_best = Gjk.gjk(mesh)
@benchmark Gjk.gjk($mesh, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  704.00 bytes
  allocs estimate:  23
  minimum time:     1.86 μs (0.00% GC)
  median time:      1.93 μs (0.00% GC)
  mean time:        2.15 μs (3.93% GC)
  maximum time:     322.45 μs (97.80% GC)

In [23]:
simplex, pt_best = Gjk.gjk(acc)
@benchmark Gjk.gjk($acc, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  704.00 bytes
  allocs estimate:  24
  minimum time:     1.11 μs (0.00% GC)
  median time:      1.28 μs (0.00% GC)
  mean time:        1.79 μs (7.19% GC)
  maximum time:     568.16 μs (98.55% GC)